<a href="https://colab.research.google.com/github/steelannelida/nanoGPT/blob/master/nanogpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/steelannelida/nanoGPT.git

Cloning into 'nanoGPT'...
remote: Enumerating objects: 703, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 703 (delta 11), reused 17 (delta 8), pack-reused 682 (from 1)
Receiving objects: 100% (703/703), 1.10 MiB | 8.36 MiB/s, done.
Resolving deltas: 100% (396/396), done.


In [2]:
!pip install torch numpy transformers datasets tiktoken wandb tqdm triton


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!yes  | python nanoGPT/data/shakespeare/prepare.py

train has 301,966 tokens
val has 36,059 tokens


In [4]:
with open('nanoGPT/data/shakespeare/input.txt') as f:
  text = f.read()

chars = sorted(set(text))
vocab_size=len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}

def encode(t):
  return [stoi[c] for c in t]

def decode(seq):
  return ''.join([chars[i] for i in seq])

decode(encode("sandwitch"))

'sandwitch'

In [5]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data = torch.tensor(encode(text), device=device)
data.shape, data.dtype

n = int(data.shape[0] * 0.9)
train_data = data[:n]
valid_data = data[n:]

In [6]:
#torch.manual_seed(1337)


def get_batch(data_set=train_data, batch_size=32, seq_length=128):
    x = torch.zeros([batch_size, seq_length], dtype=torch.int, device=device)
    y = torch.zeros([batch_size, seq_length], dtype=torch.int, device=device)
    for b in range(batch_size):
        t = torch.randint(0, data_set.shape[0] - seq_length - 1, [1])
        x[b] = data_set[t:t+seq_length]
        y[b] = data_set[t+1:t+seq_length+1]
    return x, y

x, y = get_batch()
print(decode(x[13]))
print(decode(y[13]))

 in person bear me
Like a true friar. More reasons for this action
At our more leisure shall I render you;
Only, this one: Lord 
in person bear me
Like a true friar. More reasons for this action
At our more leisure shall I render you;
Only, this one: Lord A


In [7]:
import random
arith_chars = '0123456789-=*+'
arith_idx = {c:idx for idx, c in enumerate(arith_chars)}


def gen_expression(digits=5, terms=3):
  s = ''
  for i in range(terms):
    if i > 0:
      s += random.choice('*-+')
    num = ''
    for j in range(digits):
      num += random.choice('0123456789')
    num = str(int(num))
    s += num
  return s

def gen_eq():
  expr = gen_expression()
  value = eval(expr)
  return f'{expr}={value}'

gen_eq()


'51241+5341+61932=118514'

True

In [43]:
import torch.nn as nn

class DecoderLayer(nn.Module):
  def __init__(self, embed_size=256, wide_size=1024, nheads=16, dropout=0.5):
    super().__init__()
    # self.attn = nn.MultiheadAttention(embed_size, nheads,
    #                                   batch_first=True)
    self.nheads = nheads
    self.head_size = embed_size // nheads
    self.pre_attn = nn.Linear(embed_size, 3 * embed_size)
    self.post_attn = nn.Linear(embed_size, embed_size)
    self.ln1 = nn.LayerNorm(embed_size)
    self.ln2 = nn.LayerNorm(embed_size)
    self.lin1 = nn.Linear(embed_size, wide_size)
    self.lin2 = nn.Linear(wide_size, embed_size)
    self.drop = nn.Dropout(dropout)

  def forward(self, x, mask):
    N, L, C = x.shape
    x = self.ln1(x)
    #a, w = self.attn.forward(x, x, x, attn_mask=mask, is_causal=True)
    kqv = self.pre_attn(x).view(N, L, self.nheads, 3*self.head_size).permute(0, 2, 1, 3)
    k = kqv[:,:,:,:self.head_size]
    q = kqv[:,:,:,self.head_size : 2*self.head_size]
    v = kqv[:,:,:,2*self.head_size:]
    av = nn.functional.scaled_dot_product_attention(q, k, v, mask.view(1, L, L))
    a = self.post_attn(av.permute(0, 2, 1, 3).view(N, L, C))
    x = x + self.drop(a)
    x = self.ln2(x)
    xx = self.lin1(x)
    xx = nn.functional.gelu(xx)
    y = self.lin2(xx)
    return x + y



class LM(nn.Module):
  def __init__(self, vocab_size, embed_size=256, nheads=16, max_pos=2048, num_layers=3, dropout=0.5):
    super().__init__()
    self.embeddings = nn.Embedding(vocab_size, embed_size)
    self.pos_embeddings = nn.Embedding(max_pos, embed_size)
    self.layers = [
        DecoderLayer(embed_size=embed_size, wide_size=4 * embed_size,
                     nheads=nheads, dropout=dropout)
        for _ in range(num_layers)
    ]
    for i, layer in enumerate(self.layers):
      self.add_module(f'decoder-{i}', layer)
    self.ln = nn.LayerNorm(embed_size)
    self.out = nn.Linear(embed_size, vocab_size)
    self.max_pos = max_pos
    self.to(device)

  def forward(self, idx):
    idx = torch.tensor(idx, device=device)
    l = idx.shape[-1]
    pe = self.pos_embeddings(torch.arange(0, l, device=device))
    e = self.embeddings(idx)
    e = e + pe.view(1, *pe.shape)
    mask = torch.tril(torch.ones([l, l], dtype=torch.bool, device=device))
    #print(mask)
    #print(w)
    for layer in self.layers:
      e = layer(e, mask)
    logits = self.out.forward(self.ln(e))
    return logits

  def generate(self, prompt, l):
    prompt = torch.tensor(prompt)
    pl = prompt.shape[0]
    result = torch.zeros([pl + l], dtype=torch.int, device=device)
    result[:pl] = prompt
    for i in range(l):
      logits = self.forward(result[:pl + i])
      sm = logits[:,-1].flatten().softmax(0)
      next_idx = torch.multinomial(sm, 1)
      result[i+pl] = next_idx
    return result

model = LM(vocab_size)

k=x
q=x[:,:-1]
model.forward(encode('jello'))
decode(model.generate(encode('hello'), 10))


<ipython-input-43-e743fabf5d0e>:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx = torch.tensor(idx, device=device)


'hello$!nOiRLr.k'

In [33]:
x, y = get_batch()
logits = model(x)
loss_fun = nn.CrossEntropyLoss()
loss = loss_fun(logits.permute(0, 2, 1), y.long())
loss

<ipython-input-32-bbf3a1f534d0>:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx = torch.tensor(idx, device=device)


tensor(4.3758, device='cuda:0', grad_fn=<NllLoss2DBackward0>)

In [48]:
torch.manual_seed(1337)

model = LM(vocab_size, num_layers=8)
model = torch.compile(model)
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
xv, yv = get_batch(valid_data, batch_size=256)
scaler = torch.cuda.amp.GradScaler()

<ipython-input-48-b8b286eeb8b6>:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [52]:
%%time
torch.set_float32_matmul_precision('medium')
for step in range(5000):
  x, y = get_batch(batch_size=32, seq_length=128)
  with torch.autocast(device_type='cuda', dtype=torch.float16):
    model.train()
    logits = model.forward(x)
    loss = loss_fun(logits.permute(0, 2, 1), y.long())

  scaler.scale(loss).backward()
  scaler.step(opt)
  scaler.update()
  # loss.backward()
  # opt.step()
  # opt.zero_grad()

  if step % 10 == 0:
    with torch.no_grad():
      with torch.autocast(device_type='cuda', dtype=torch.float16):
        model.eval()
        vlogits = model.forward(xv)
        vloss = loss_fun(vlogits.permute(0,2,1), yv.long())
      print('%d\t%f\t%f'%(step, loss, vloss))


0	2.330084	2.321952
10	2.240674	2.292088
20	2.237633	2.282294
30	2.224840	2.262885
40	2.251450	2.256018
50	2.207198	2.234732
60	2.207946	2.214935
70	2.149199	2.189362
80	2.131125	2.168802
90	2.147406	2.160749
100	2.092362	2.128591
110	2.067341	2.116857
120	2.055706	2.110833
130	2.046234	2.099337
140	2.004681	2.089618
150	2.047931	2.085074
160	2.035627	2.087979
170	1.977736	2.072329
180	2.005031	2.046731
190	1.968908	2.029851
200	1.973555	2.017879
210	1.946047	2.016523
220	1.918499	2.008947
230	1.878981	2.003235
240	1.906580	1.993696
250	1.901069	1.983843
260	1.886767	1.956607
270	1.860534	1.964939
280	1.843090	1.956652
290	1.867823	1.941780
300	1.785640	1.946332
310	1.789663	1.940130
320	1.835106	1.942779
330	1.833072	1.935559
340	1.858463	1.915943
350	1.798697	1.925139
360	1.774139	1.904985
370	1.756099	1.887990
380	1.807185	1.892697
390	1.772618	1.887367
400	1.780906	1.880911
410	1.778962	1.878105
420	1.731223	1.875539
430	1.740290	1.867718
440	1.700108	1.851022
450	1.678572	1.850341

In [53]:
with torch.profiler.profile(with_stack=True, profile_memory=True) as prof:
  x, y = get_batch(batch_size=32, seq_length=128)
  with torch.autocast(device_type='cuda', dtype=torch.float16):
    model.train()
    logits = model.forward(x)
    loss = loss_fun(logits.permute(0, 2, 1), y.long())

  loss.backward()
  opt.step()
  opt.zero_grad()


print(prof.export_chrome_trace('trace3.json'))


None


In [ ]:
!ls -lh trace2.json

-rw-r--r-- 1 root root 6.2M Oct 31 15:41 trace2.json


In [55]:
with torch.no_grad():
  with torch.autocast(device_type='cuda', dtype=torch.float16):
    model.eval()
    print(decode(model.generate(encode('To dream'), 256)))


<ipython-input-43-e743fabf5d0e>:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx = torch.tensor(idx, device=device)


To dream to castle,
And then let the Thortands o' the conquerors,
Orgiving them speaking on the temper, and kell'd the breast,
Whicomes fouly, caratuly omakin-cowagininininin: ninas omay Sas honine lhamebaponin ininedes ulys, tyines I pavininoraperes ty te'sarines


In [ ]:
import matplotlib.pyplot as plt

plt.imshow(model.embeddings.weight.detach().cpu().numpy())
# plt.plot(model.pos_embeddings.weight[:,37].detach().cpu().numpy())
# plt.plot(model.pos_embeddings.weight[:,4].detach().cpu().numpy())


In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE()
y = tsne.fit_transform(model.out.weight.detach().cpu().numpy())

plt.scatter(y[:,0], y[:,1], s=1, alpha=0.1)
for i, c in enumerate(chars):
  plt.text(y[i,0], y[i,1], c)

In [ ]:
model.out.weight.shape